# using the PON coverage, calculate the coverage difference of tumor samples 

In [1]:
# get the code
import sys
sys.path.append('../code')
from coverage import get_coverage

### prepare the files

In [13]:
# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
tooldata = f"{home}/Dropbox/Icke/Work/somVar/tooldata"

cnvdata = f"{tooldata}/myCNVdata"

output_path = f"{cnvdata}/output"

In [14]:
cnvdata

'/Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata'

In [24]:
sample = "01_A"
chrom = "chr7"
sample_cov_file = f"{cnvdata}/cnv/{sample}.{chrom}.cov"
pon_cov_file = f"{output_path}/PONcoverage/PON_coverage.{chrom}.removed.csv"
sample_df = pd.read_csv(sample_cov_file, sep='\t', compression="gzip")
pon_df = pd.read_csv(pon_cov_file, sep='\t').loc[:,['Chr', 'Pos', 'ExonPos', 'meanCov', 'medianCov', 'std']]
sample_df = sample_df.merge(pon_df, on=['Chr', 'Pos', 'ExonPos'], how="inner").loc[:,['Chr', 'Pos', 'ExonPos', 'Coverage','meanCov', 'medianCov', 'std']]
sample_df

Chr        Pos  ExonPos  Coverage     meanCov   medianCov        std
0      chr7     208900     2041    340.27   62.506986   62.506986   8.098135
1      chr7     208950     2091    321.96   59.648168   59.798555   7.916760
2      chr7     209000     2141    221.63   37.648584   37.493645   6.896189
3      chr7     246500     2302    363.31   96.456188   96.716537  17.548990
4      chr7     248300     2339    658.75  111.014898  111.014898  15.119444
...     ...        ...      ...       ...         ...         ...        ...
29875  chr7  159144250  2422935    199.50   40.974579   39.189834  10.229816
29876  chr7  159144700  2423384    255.36   45.594039   45.594039  15.218426
29877  chr7  159144750  2423434    209.75   38.186303   38.186303  12.604047
29878  chr7  159144800  2423484    175.55   30.157172   30.157172  10.423255
29879  chr7  159144850  2423534    127.00   19.179232   19.108572   7.251982

[29880 rows x 7 columns]

In [91]:
cov_df['ploidy'] = cov_df['normCov'] / cov_df['PONmeanCov'] * 2
cov_df.query('ploidy > 100')

Chr        Pos  Coverage  ExonPos   normCov  PONmeanCov    PONstd  \
14637  chr7   63558600      0.28   734206  0.115322    0.000000  0.000000   
14638  chr7   63558650      1.27   734256  0.523068    0.000000  0.000000   
17583  chr7   74736650      0.28   890591  0.115322    0.000000  0.000000   
29288  chr7  102560750      0.10  1492453  0.041186    0.000000  0.000000   
29307  chr7  102567950      0.47  1493407  0.193576    0.000000  0.000000   
29447  chr7  102690450      0.93  1504604  0.383034    0.000000  0.000000   
39830  chr7  142038500      1.88  2030881  0.774305    0.009261  0.047220   
40138  chr7  142870450      0.06  2084479  0.024712    0.000000  0.000000   
41061  chr7  144183600      0.77  2149014  0.317136    0.003504  0.017865   

           ploidy  
14637         inf  
14638         inf  
17583         inf  
29288         inf  
29307         inf  
29447         inf  
39830  167.224732  
40138         inf  
41061  181.034222

### filter out certain stuff before applying rolling average

In [98]:
select_df = cov_df.query('ploidy < 100')
select_df['rollingPloidy'] = select_df['ploidy'].rolling(500).mean()
select_df.to_csv(f"{cnvdata}/output/{bam_file}.diff.csv", sep='\t', index=False)

<ipython-input-98-946f9ca0fd86>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_df['rollingPloidy'] = select_df['ploidy'].rolling(500).mean()
